# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [2]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import datetime
from skimage import io
from skimage.transform import resize 
import matplotlib.pyplot as plt
import imageio 

In [21]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Dropout, Flatten, TimeDistributed, BatchNormalization, Activation, Conv3D, MaxPooling3D, Rescaling
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from tensorflow.keras.regularizers import l2
from keras.applications.vgg16 import VGG16

In [3]:
#pip install scikit-image

We set the random seed so that the results don't vary drastically.

In [4]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
train_doc = np.random.permutation(open('C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('C:/Per/IIITB_Data/GestureRecognition_data/Project_data/val.csv').readlines())

In [26]:
import pandas as pd
A=['FolderName','Action','Class']
data = [(train_doc[i].strip().split(';')) for i in range(0, len(train_doc))]
df=pd.DataFrame(data, columns=A)
df.head()

,FolderName,Action,Class
0,WIN_20180926_16_54_08_Pro_Right_Swipe_new,Right_Swipe_new,1
1,WIN_20180925_18_02_58_Pro_Thumbs_Down_new,Thumbs_Down_new,3
2,WIN_20180925_17_33_08_Pro_Left_Swipe_new,Left_Swipe_new,0
3,WIN_20180925_17_51_17_Pro_Thumbs_Up_new,Thumbs_Up_new,4
4,WIN_20180926_17_17_35_Pro_Left_Swipe_new,Left_Swipe_new,0


In [27]:
print(df.shape)
print(df[['Class']].value_counts())
print(df[['Class','Action']].value_counts())

(663, 3)
Class
1        137
3        137
0        136
2        130
4        123
dtype: int64
Class  Action                       
1      Right_Swipe_new                  103
3      Thumbs_Down_new                  100
0      Left_Swipe_new                    96
2      Stop_new                          93
4      Thumbs_Up_new                     87
0      Left Swipe_new_Left Swipe_new     40
2      Stop Gesture_new                  37
3      Thumbs Down_new                   37
4      Thumbs Up_new                     36
1      Right Swipe_new                   34
dtype: int64


In [15]:
curr_dt_time = datetime.datetime.now()
train_path = 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train'
val_path = 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
steps_per_epoch = 0 
validation_steps = 0

# training sequences = 663
# validation sequences = 100


In [17]:
def set_steps_epoch_validation(l_batch_size):
    global steps_per_epoch, validation_steps
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/l_batch_size)
    else:
        steps_per_epoch = (num_train_sequences//l_batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/l_batch_size)
    else:
        validation_steps = (num_val_sequences//l_batch_size) + 1
    print ('# steps per epoch =', steps_per_epoch)
    print ('# validation steps =', validation_steps)
    

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with some of the parts of the generator function such that you get high accuracy.

In [8]:
#plt.imshow

In [134]:
def generator(source_path, folder_list, batch_size, num_epochs, no_of_frames=15, height = 120, width = 120):
    print( 'Source path = ', source_path, '; batch size =', batch_size, '; epochs =', num_epochs, '; no of frames =', no_of_frames, '; height =', height, '; width =', width)
    if no_of_frames == 25:
        img_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,16,18,20,22,23,24,25,26,27,28,29]
    if no_of_frames == 20:
        img_idx = [0,1,2,4,6,7,8,9,10,12,14,16,18,20,22,24,26,27,28,29]
    elif no_of_frames == 18:
        img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    elif no_of_frames == 15:
        img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28]
    else:
        print('Invalid frame count: ', no_of_frames)
        return
        
    #img_idx = [0,4,6,8,10,12,14,16,18,20,22,24]
    while True:
        t = np.random.permutation(folder_list)
        #print("Permutation=>",len(t),"=>",t[0])
        num_batches = int(len(t)/batch_size)
        #print("No of batches=>",num_batches)
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size,no_of_frames,height,width,3))
            #print("batch data=>",batch_data[0][0][0][0][3])
            batch_labels = np.zeros((batch_size,5))
            #print("batch labels=>", batch_labels[0])
            for folder in range(batch_size):
                #print(folder)
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                #print((imgs))
                for idx,item in enumerate(img_idx):
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #print(image.shape)
                    #plt.imshow(image)
                    #plt.show()
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(height,width)).astype(np.float32)
                    else:
                        image = resize(image,(height,width)).astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,no_of_frames,height,width,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = io.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(height,width)).astype(np.float32)
                    else:
                        image = resize(image,(height,width)).astype(np.float32)

                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

## Model
Here you make the model using different functionalities that Keras provides. You might want to use `TimeDistributed`, `GRU` and other RNN structures after doing transfer learning. Also remember that the last layer is the softmax. Remember that the network is designed in such a way that the model is able to fit in the memory of the webcam.

# Transfer Learning : CNN + RNN

#### Transfer learning using VGG16 & RNN (GRU)

In [18]:
batch_size = 30
print ('# batch size =', batch_size)
num_epochs = 30
print ('# num epochs =', num_epochs)
no_of_frames = 15
print ('# num of frames =', no_of_frames)
height = 120
width = 120
print ('# hight =', height, '; width =', width)
set_steps_epoch_validation(batch_size)
train_generator = generator(train_path, train_doc, batch_size, num_epochs)
val_generator = generator(val_path, val_doc, batch_size, num_epochs)

# batch size = 30
# num epochs = 30
# num of frames = 15
# hight = 120 ; width = 120
# steps per epoch = 23
# validation steps = 4


In [24]:
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(height,width,3))
x = base_model.output
x = Flatten()(x)
#x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_model = Model(inputs=base_model.input, outputs=features)
    
for layer in base_model.layers:
    layer.trainable = False
        
model = Sequential()
model.add(TimeDistributed(conv_model, input_shape=(no_of_frames,height,width,3)))
model.add(GRU(32, return_sequences=True))
model.add(GRU(16))
model.add(Dropout(0.5))
model.add(Dense(8, activation='relu'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [13]:
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 15, 64)            15009664  
 ributed)                                                        
                                                                 
 gru (GRU)                   (None, 15, 32)            9408      
                                                                 
 gru_1 (GRU)                 (None, 16)                2400      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 5)                 45        
                                                        

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [14]:
model_name = 'model_init_conv_lstm_VGG16' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
#filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath = model_name + 'model_VGG.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [15]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train ; batch size = 30 ; epochs = 30
Epoch 1/30
23/23 [==============================] - 357s 16s/step - loss: 1.7202 - categorical_accuracy: 0.1840 - val_loss: 1.6611 - val_categorical_accuracy: 0.1900 - lr: 0.0010
Epoch 2/30
23/23 [==============================] - 395s 17s/step - loss: 1.6166 - categorical_accuracy: 0.2262 - val_loss: 1.6525 - val_categorical_accuracy: 0.2000 - lr: 0.0010
Epoch 3/30
23/23 [==============================] - 399s 17s/step - loss: 1.5370 - categorical_accuracy: 0.2730 - val_loss: 1.5789 - val_categorical_accuracy: 0.2700 - lr: 0.0010
Epoch 4/30
21/23 [==========================>...] - ETA: 30s - loss: 1.5072 - categorical_accuracy: 0.3095WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 397s 17s/step - loss: 1.5055 - categorical_accuracy: 0.3137 - val_loss: 1.5064 - val_categorical_accuracy: 0.2600 - lr: 0.0010
Epoch 5/30
23/23 [==============================] - 396s 17s/step - loss: 1.4793 - categorical_accuracy: 0.3198 - val_loss: 1.4858 - val_categorical_accuracy: 0.3100 - lr: 0.0010
Epoch 6/30
21/23 [==========================>...] - ETA: 31s - loss: 1.4286 - categorical_accuracy: 0.3984WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 394s 17s/step - loss: 1.4284 - categorical_accuracy: 0.3952 - val_loss: 1.4687 - val_categorical_accuracy: 0.3200 - lr: 0.0010
Epoch 7/30
23/23 [==============================] - 371s 16s/step - loss: 1.3896 - categorical_accuracy: 0.4299 - val_loss: 1.4609 - val_categorical_accuracy: 0.3700 - lr: 0.0010
Epoch 8/30
21/23 [==========================>...] - ETA: 28s - loss: 1.3665 - categorical_accuracy: 0.4413WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 371s 16s/step - loss: 1.3669 - categorical_accuracy: 0.4389 - val_loss: 1.4184 - val_categorical_accuracy: 0.4100 - lr: 0.0010
Epoch 9/30
23/23 [==============================] - 400s 18s/step - loss: 1.3086 - categorical_accuracy: 0.4766 - val_loss: 1.4174 - val_categorical_accuracy: 0.3600 - lr: 0.0010
Epoch 10/30
21/23 [==========================>...] - ETA: 31s - loss: 1.2971 - categorical_accuracy: 0.4778WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 399s 17s/step - loss: 1.3026 - categorical_accuracy: 0.4766 - val_loss: 1.3891 - val_categorical_accuracy: 0.3900 - lr: 0.0010
Epoch 11/30
23/23 [==============================] - 402s 18s/step - loss: 1.2129 - categorical_accuracy: 0.5354 - val_loss: 1.3132 - val_categorical_accuracy: 0.4600 - lr: 0.0010
Epoch 12/30
21/23 [==========================>...] - ETA: 31s - loss: 1.1845 - categorical_accuracy: 0.5746WARNING:tensorflow:Can save best model only with val_loss available, skipping.


...

23/23 [==============================] - 400s 17s/step - loss: 0.8469 - categorical_accuracy: 0.7511 - val_loss: 1.0700 - val_categorical_accuracy: 0.5000 - lr: 0.0010
Epoch 23/30
23/23 [==============================] - 414s 18s/step - loss: 0.8183 - categorical_accuracy: 0.7451 - val_loss: 1.0894 - val_categorical_accuracy: 0.5200 - lr: 0.0010
Epoch 24/30
21/23 [==========================>...] - ETA: 32s - loss: 0.7930 - categorical_accuracy: 0.7619WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 411s 18s/step - loss: 0.7908 - categorical_accuracy: 0.7632 - val_loss: 1.0477 - val_categorical_accuracy: 0.5500 - lr: 0.0010
Epoch 25/30
23/23 [==============================] - 389s 17s/step - loss: 0.7556 - categorical_accuracy: 0.7888 - val_loss: 1.0342 - val_categorical_accuracy: 0.5400 - lr: 0.0010
Epoch 26/30
21/23 [==========================>...] - ETA: 29s - loss: 0.7381 - categorical_accuracy: 0.8016WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 379s 17s/step - loss: 0.7431 - categorical_accuracy: 0.8024 - val_loss: 0.9884 - val_categorical_accuracy: 0.5700 - lr: 0.0010
Epoch 27/30
23/23 [==============================] - 402s 18s/step - loss: 0.7255 - categorical_accuracy: 0.8130 - val_loss: 1.0411 - val_categorical_accuracy: 0.5300 - lr: 0.0010
Epoch 28/30
21/23 [==========================>...] - ETA: 31s - loss: 0.7121 - categorical_accuracy: 0.8159WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 408s 18s/step - loss: 0.7117 - categorical_accuracy: 0.8175 - val_loss: 0.9877 - val_categorical_accuracy: 0.6400 - lr: 0.0010
Epoch 29/30
23/23 [==============================] - 405s 18s/step - loss: 0.7041 - categorical_accuracy: 0.8039 - val_loss: 0.9556 - val_categorical_accuracy: 0.6400 - lr: 0.0010
Epoch 30/30
21/23 [==========================>...] - ETA: 31s - loss: 0.7060 - categorical_accuracy: 0.8032WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 406s 18s/step - loss: 0.6995 - categorical_accuracy: 0.8100 - val_loss: 0.9641 - val_categorical_accuracy: 0.5800 - lr: 0.0010


In [16]:
#model.save(filepath)

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


#### Transfer Learning Using Resnet50

In [17]:
batch_size = 30
print ('# batch size =', batch_size)
num_epochs = 30
print ('# num epochs =', num_epochs)
no_of_frames = 15
print ('# num of frames =', no_of_frames)
height = 120
width = 120
print ('# hight =', height, '; width =', width)
set_steps_epoch_validation(batch_size)
train_generator = generator(train_path, train_doc, batch_size, num_epochs)
val_generator = generator(val_path, val_doc, batch_size, num_epochs)

# batch size = 30
# num epochs = 30
# num of frames = 15
# hight = 120 ; width = 120
# steps per epoch = 23
# validation steps = 4


In [30]:
base_resnet50_model = tf.keras.applications.resnet50.ResNet50(include_top=False, #Do not include FC layer at the end
                                          input_shape=(height,width,3),
                                          weights='imagenet')
x = base_resnet50_model.output
x = Flatten()(x)
#x.add(Dropout(0.5))
features = Dense(64, activation='relu')(x)
conv_resnet50_model = Model(inputs=base_resnet50_model.input, outputs=features)
    
for layer in base_resnet50_model.layers:
    layer.trainable = False

In [12]:
model_resnet50 = Sequential()
model_resnet50.add(TimeDistributed(conv_resnet50_model, input_shape=(15,height,width,3)))
model_resnet50.add(GRU(32, return_sequences=True))
model_resnet50.add(GRU(16))
model_resnet50.add(Dropout(0.5))
model_resnet50.add(Dense(8, activation='relu'))
model_resnet50.add(Dense(5, activation='softmax'))

In [13]:
sgd_resnet50 = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model_resnet50.compile(optimizer=sgd_resnet50, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_resnet50.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 15, 64)            25684928  
 ributed)                                                        
                                                                 
 gru (GRU)                   (None, 15, 32)            9408      
                                                                 
 gru_1 (GRU)                 (None, 16)                2400      
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 5)                 45        
                                                        

In [14]:
model_resnet50_name = 'model_init_conv_lstm_resnet50' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_resnet50_name):
    os.mkdir(model_resnet50_name)
        
#filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath_resnet50 = model_resnet50_name + 'model_resnet50.h5'

checkpoint_resnet50 = ModelCheckpoint(filepath_resnet50, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)

LR_resnet50 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list_resnet50 = [checkpoint_resnet50, LR_resnet50]

##### Using 20 epochs & 15 frames

In [24]:
model_resnet50.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list_resnet50, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train ; batch size = 30 ; epochs = 20
Epoch 1/20
23/23 [==============================] - 161s 7s/step - loss: 1.6147 - categorical_accuracy: 0.2278 - val_loss: 1.4702 - val_categorical_accuracy: 0.3300 - lr: 0.0010
Epoch 2/20
23/23 [==============================] - 176s 8s/step - loss: 1.4441 - categorical_accuracy: 0.3167 - val_loss: 1.3014 - val_categorical_accuracy: 0.4500 - lr: 0.0010
Epoch 3/20
23/23 [==============================] - 184s 8s/step - loss: 1.3122 - categorical_accuracy: 0.4329 - val_loss: 1.1610 - val_categorical_accuracy: 0.6300 - lr: 0.0010
Epoch 4/20
21/23 [==========================>...] - ETA: 14s - loss: 1.1961 - categorical_accuracy: 0.5016WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 183s 8s/step - loss: 1.1989 - categorical_accuracy: 0.5023 - val_loss: 1.0818 - val_categorical_accuracy: 0.7300 - lr: 0.0010
Epoch 5/20
23/23 [==============================] - 186s 8s/step - loss: 1.1226 - categorical_accuracy: 0.5686 - val_loss: 1.0277 - val_categorical_accuracy: 0.7700 - lr: 0.0010
Epoch 6/20
21/23 [==========================>...] - ETA: 14s - loss: 1.0263 - categorical_accuracy: 0.6635WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 184s 8s/step - loss: 1.0341 - categorical_accuracy: 0.6531 - val_loss: 1.0084 - val_categorical_accuracy: 0.8000 - lr: 0.0010
Epoch 7/20
23/23 [==============================] - 187s 8s/step - loss: 0.9537 - categorical_accuracy: 0.6893 - val_loss: 0.9367 - val_categorical_accuracy: 0.8000 - lr: 0.0010
Epoch 8/20
21/23 [==========================>...] - ETA: 14s - loss: 0.9075 - categorical_accuracy: 0.7508WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 186s 8s/step - loss: 0.9054 - categorical_accuracy: 0.7496 - val_loss: 0.8078 - val_categorical_accuracy: 0.8800 - lr: 0.0010
Epoch 9/20
23/23 [==============================] - 186s 8s/step - loss: 0.8360 - categorical_accuracy: 0.8100 - val_loss: 0.8340 - val_categorical_accuracy: 0.8500 - lr: 0.0010
Epoch 10/20
21/23 [==========================>...] - ETA: 14s - loss: 0.7785 - categorical_accuracy: 0.8111WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 182s 8s/step - loss: 0.7821 - categorical_accuracy: 0.8024 - val_loss: 0.7897 - val_categorical_accuracy: 0.8600 - lr: 0.0010
Epoch 11/20
23/23 [==============================] - 187s 8s/step - loss: 0.7460 - categorical_accuracy: 0.8431 - val_loss: 0.7939 - val_categorical_accuracy: 0.8500 - lr: 0.0010
Epoch 12/20
21/23 [==========================>...] - ETA: 14s - loss: 0.7510 - categorical_accuracy: 0.8286WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 182s 8s/step - loss: 0.7519 - categorical_accuracy: 0.8311 - val_loss: 0.7006 - val_categorical_accuracy: 0.9200 - lr: 0.0010
Epoch 13/20
23/23 [==============================] - 185s 8s/step - loss: 0.7154 - categorical_accuracy: 0.8537 - val_loss: 0.7272 - val_categorical_accuracy: 0.8800 - lr: 0.0010
Epoch 14/20
21/23 [==========================>...] - ETA: 14s - loss: 0.6820 - categorical_accuracy: 0.8746WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - ETA: 0s - loss: 0.6824 - categorical_accuracy: 0.8733
Epoch 14: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
23/23 [==============================] - 183s 8s/step - loss: 0.6824 - categorical_accuracy: 0.8733 - val_loss: 0.7095 - val_categorical_accuracy: 0.8800 - lr: 0.0010
Epoch 15/20
23/23 [==============================] - 186s 8s/step - loss: 0.6698 - categorical_accuracy: 0.8582 - val_loss: 0.6677 - val_categorical_accuracy: 0.8800 - lr: 5.0000e-04
Epoch 16/20
20/23 [=========================>....] - ETA: 21s - loss: 0.6319 - categorical_accuracy: 0.9033WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 182s 8s/step - loss: 0.6302 - categorical_accuracy: 0.9020 - val_loss: 0.6722 - val_categorical_accuracy: 0.8500 - lr: 5.0000e-04
Epoch 17/20
23/23 [==============================] - 184s 8s/step - loss: 0.6403 - categorical_accuracy: 0.8808 - val_loss: 0.6483 - val_categorical_accuracy: 0.9000 - lr: 5.0000e-04
Epoch 18/20
20/23 [=========================>....] - ETA: 21s - loss: 0.6266 - categorical_accuracy: 0.8917WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 184s 8s/step - loss: 0.6288 - categorical_accuracy: 0.8929 - val_loss: 0.6867 - val_categorical_accuracy: 0.9000 - lr: 5.0000e-04
Epoch 19/20
23/23 [==============================] - 185s 8s/step - loss: 0.6119 - categorical_accuracy: 0.8944 - val_loss: 0.6301 - val_categorical_accuracy: 0.8900 - lr: 5.0000e-04
Epoch 20/20
20/23 [=========================>....] - ETA: 21s - loss: 0.5924 - categorical_accuracy: 0.9083WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 182s 8s/step - loss: 0.5930 - categorical_accuracy: 0.9125 - val_loss: 0.5982 - val_categorical_accuracy: 0.9100 - lr: 5.0000e-04


###### On 28th epoch we have got best metric score as categorical accuracy = 91.25% and val_categorical_accuracy = 91%

##### Using 30 epochs & 15 frames

In [15]:
model_resnet50.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list_resnet50, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train ; batch size = 30 ; epochs = 30
Epoch 1/30
23/23 [==============================] - 217s 9s/step - loss: 1.5908 - categorical_accuracy: 0.2866 - val_loss: 1.4696 - val_categorical_accuracy: 0.3800 - lr: 0.0010
Epoch 2/30
23/23 [==============================] - 162s 7s/step - loss: 1.3820 - categorical_accuracy: 0.4314 - val_loss: 1.2498 - val_categorical_accuracy: 0.5600 - lr: 0.0010
Epoch 3/30
23/23 [==============================] - 168s 7s/step - loss: 1.2378 - categorical_accuracy: 0.5143 - val_loss: 1.1940 - val_categorical_accuracy: 0.6100 - lr: 0.0010
Epoch 4/30
21/23 [==========================>...] - ETA: 13s - loss: 1.1214 - categorical_accuracy: 0.5968WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 172s 8s/step - loss: 1.1224 - categorical_accuracy: 0.5988 - val_loss: 1.0749 - val_categorical_accuracy: 0.6600 - lr: 0.0010
Epoch 5/30
23/23 [==============================] - 172s 8s/step - loss: 1.0138 - categorical_accuracy: 0.6471 - val_loss: 1.0308 - val_categorical_accuracy: 0.6700 - lr: 0.0010
Epoch 6/30
21/23 [==========================>...] - ETA: 13s - loss: 1.0048 - categorical_accuracy: 0.6587WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 170s 7s/step - loss: 0.9948 - categorical_accuracy: 0.6637 - val_loss: 0.9867 - val_categorical_accuracy: 0.6900 - lr: 0.0010
Epoch 7/30
23/23 [==============================] - 172s 8s/step - loss: 0.8924 - categorical_accuracy: 0.7345 - val_loss: 0.9765 - val_categorical_accuracy: 0.7200 - lr: 0.0010
Epoch 8/30
21/23 [==========================>...] - ETA: 13s - loss: 0.8582 - categorical_accuracy: 0.7667WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 170s 7s/step - loss: 0.8542 - categorical_accuracy: 0.7677 - val_loss: 0.9684 - val_categorical_accuracy: 0.7100 - lr: 0.0010
Epoch 9/30
23/23 [==============================] - 173s 8s/step - loss: 0.8190 - categorical_accuracy: 0.7722 - val_loss: 0.9405 - val_categorical_accuracy: 0.7100 - lr: 0.0010
Epoch 10/30
21/23 [==========================>...] - ETA: 13s - loss: 0.7706 - categorical_accuracy: 0.8111WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - 169s 7s/step - loss: 0.7699 - categorical_accuracy: 0.8084 - val_loss: 0.9207 - val_categorical_accuracy: 0.7100 - lr: 0.0010
Epoch 11/30
23/23 [==============================] - 171s 7s/step - loss: 0.7388 - categorical_accuracy: 0.8115 - val_loss: 0.8241 - val_categorical_accuracy: 0.7700 - lr: 0.0010
Epoch 12/30
21/23 [==========================>...] - ETA: 13s - loss: 0.7064 - categorical_accuracy: 0.8349WARNING:tensorflow:Can save best model only with val_loss available, skipping.


...

23/23 [==============================] - ETA: 0s - loss: 0.5351 - categorical_accuracy: 0.8974
Epoch 20: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
23/23 [==============================] - 170s 7s/step - loss: 0.5351 - categorical_accuracy: 0.8974 - val_loss: 0.6642 - val_categorical_accuracy: 0.8500 - lr: 2.5000e-04
Epoch 21/30
23/23 [==============================] - 174s 8s/step - loss: 0.5628 - categorical_accuracy: 0.8869 - val_loss: 0.6766 - val_categorical_accuracy: 0.8500 - lr: 1.2500e-04
Epoch 22/30
18/23 [======================>.......] - ETA: 32s - loss: 0.5304 - categorical_accuracy: 0.9056WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - ETA: 0s - loss: 0.5224 - categorical_accuracy: 0.9080
Epoch 22: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
23/23 [==============================] - 170s 7s/step - loss: 0.5224 - categorical_accuracy: 0.9080 - val_loss: 0.6627 - val_categorical_accuracy: 0.8500 - lr: 1.2500e-04
Epoch 23/30
23/23 [==============================] - 173s 8s/step - loss: 0.5233 - categorical_accuracy: 0.8959 - val_loss: 0.6733 - val_categorical_accuracy: 0.8400 - lr: 6.2500e-05
Epoch 24/30
17/23 [=====================>........] - ETA: 39s - loss: 0.5458 - categorical_accuracy: 0.9020WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - ETA: 0s - loss: 0.5444 - categorical_accuracy: 0.9020
Epoch 24: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
23/23 [==============================] - 171s 7s/step - loss: 0.5444 - categorical_accuracy: 0.9020 - val_loss: 0.7211 - val_categorical_accuracy: 0.8100 - lr: 6.2500e-05
Epoch 25/30
23/23 [==============================] - 173s 8s/step - loss: 0.5297 - categorical_accuracy: 0.8974 - val_loss: 0.6711 - val_categorical_accuracy: 0.8300 - lr: 3.1250e-05
Epoch 26/30
16/23 [===================>..........] - ETA: 46s - loss: 0.5450 - categorical_accuracy: 0.9146WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - ETA: 0s - loss: 0.5212 - categorical_accuracy: 0.9231
Epoch 26: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
23/23 [==============================] - 173s 8s/step - loss: 0.5212 - categorical_accuracy: 0.9231 - val_loss: 0.7037 - val_categorical_accuracy: 0.8200 - lr: 3.1250e-05
Epoch 27/30
23/23 [==============================] - 173s 8s/step - loss: 0.5397 - categorical_accuracy: 0.9035 - val_loss: 0.6690 - val_categorical_accuracy: 0.8700 - lr: 1.5625e-05
Epoch 28/30
15/23 [==================>...........] - ETA: 52s - loss: 0.5268 - categorical_accuracy: 0.9089WARNING:tensorflow:Can save best model only with val_loss available, skipping.


23/23 [==============================] - ETA: 0s - loss: 0.5303 - categorical_accuracy: 0.9140
Epoch 28: ReduceLROnPlateau reducing learning rate to 1e-05.
23/23 [==============================] - 172s 8s/step - loss: 0.5303 - categorical_accuracy: 0.9140 - val_loss: 0.6422 - val_categorical_accuracy: 0.8900 - lr: 1.5625e-05
Epoch 29/30
23/23 [==============================] - 174s 8s/step - loss: 0.5352 - categorical_accuracy: 0.9020 - val_loss: 0.6679 - val_categorical_accuracy: 0.8700 - lr: 1.0000e-05
Epoch 30/30
23/23 [==============================] - 172s 8s/step - loss: 0.5358 - categorical_accuracy: 0.9065 - val_loss: 0.7304 - val_categorical_accuracy: 0.8400 - lr: 1.0000e-05


###### On 28th epoch we have got best metric score as categorical accuracy = 91.40% and val_categorical_accuracy = 89%

In [16]:
#model_resnet50.save(filepath_resnet50)

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


##### Using 30 epochs & 20 frames

In [19]:
batch_size = 30
print ('# batch size =', batch_size)
num_epochs = 30
print ('# num epochs =', num_epochs)
no_of_frames = 20
print ('# num of frames =', no_of_frames)
height = 120
width = 120
print ('# hight =', height, '; width =', width)
set_steps_epoch_validation(batch_size)
train_generator = generator(train_path, train_doc, batch_size, num_epochs, no_of_frames)
val_generator = generator(val_path, val_doc, batch_size, num_epochs, no_of_frames)

# batch size = 30
# num epochs = 30
# num of frames = 20
# hight = 120 ; width = 120
# steps per epoch = 23
# validation steps = 4


In [36]:
model_resnet50_20F = Sequential()
model_resnet50_20F.add(TimeDistributed(conv_resnet50_model, input_shape=(no_of_frames,height,width,3)))
model_resnet50_20F.add(GRU(32, return_sequences=True))
model_resnet50_20F.add(GRU(16))
model_resnet50_20F.add(Dropout(0.5))
model_resnet50_20F.add(Dense(8, activation='relu'))
model_resnet50_20F.add(Dense(5, activation='softmax'))

In [37]:
sgd_resnet50 = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model_resnet50_20F.compile(optimizer=sgd_resnet50, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_resnet50_20F.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDi  (None, 20, 64)            25684928  
 stributed)                                                      
                                                                 
 gru_6 (GRU)                 (None, 20, 32)            9408      
                                                                 
 gru_7 (GRU)                 (None, 16)                2400      
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_9 (Dense)             (None, 8)                 136       
                                                                 
 dense_10 (Dense)            (None, 5)                 45        
                                                      

In [38]:
model_resnet50_name = 'model_init_conv_lstm_resnet50' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_resnet50_name):
    os.mkdir(model_resnet50_name)
        
#filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
filepath_resnet50 = model_resnet50_name + 'model-{epoch:05d}-{categorical_accuracy:.5f}.h5'

checkpoint_resnet50 = ModelCheckpoint(filepath_resnet50, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

LR_resnet50 = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list_resnet50 = [checkpoint_resnet50, LR_resnet50]

In [39]:
model_resnet50_20F.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list_resnet50, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train ; batch size = 30 ; epochs = 30 no of frames = 20
Epoch 1/30

Epoch 1: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00001-0.13333.h5
 1/23 [>.............................] - ETA: 4:55 - loss: 1.8425 - categorical_accuracy: 0.1333
Epoch 1: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00001-0.16667.h5
 2/23 [=>............................] - ETA: 3:40 - loss: 1.8118 - categorical_accuracy: 0.1667
Epoch 1: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00001-0.20000.h5
 3/23 [==>...........................] - ETA: 3:37 - loss: 1.7850 - categorical_accuracy: 0.2000
Epoch 1: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00001-0.21667.h5
 4/23 [====>.........................] - ETA: 3:28 - loss: 1.7331 - categorical_accuracy: 0.2167
Epoch 1: saving model to model_init_conv_lstm_resnet50_202

10/23 [============>.................] - ETA: 1:59 - loss: 1.0988 - categorical_accuracy: 0.6067
Epoch 4: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00004-0.61212.h5
11/23 [=============>................] - ETA: 1:50 - loss: 1.0927 - categorical_accuracy: 0.6121
Epoch 4: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00004-0.60833.h5
12/23 [==============>...............] - ETA: 1:40 - loss: 1.0880 - categorical_accuracy: 0.6083
Epoch 4: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00004-0.61538.h5
13/23 [===============>..............] - ETA: 1:31 - loss: 1.0832 - categorical_accuracy: 0.6154
Epoch 4: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00004-0.61905.h5
14/23 [=================>............] - ETA: 1:22 - loss: 1.0808 - categorical_accuracy: 0.6190
Epoch 4: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00004-0.6155

 4/23 [====>.........................] - ETA: 2:54 - loss: 0.9530 - categorical_accuracy: 0.7583
Epoch 6: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00006-0.75333.h5
 5/23 [=====>........................] - ETA: 2:46 - loss: 0.9448 - categorical_accuracy: 0.7533
Epoch 6: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00006-0.73889.h5
 6/23 [======>.......................] - ETA: 2:35 - loss: 0.9318 - categorical_accuracy: 0.7389
Epoch 6: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00006-0.73333.h5
 7/23 [========>.....................] - ETA: 2:27 - loss: 0.9228 - categorical_accuracy: 0.7333
Epoch 6: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00006-0.72500.h5
 8/23 [=========>....................] - ETA: 2:17 - loss: 0.9270 - categorical_accuracy: 0.7250
Epoch 6: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00006-0.7111

15/23 [==================>...........] - ETA: 1:17 - loss: 0.7717 - categorical_accuracy: 0.8200
Epoch 9: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00009-0.82292.h5
16/23 [===================>..........] - ETA: 1:07 - loss: 0.7655 - categorical_accuracy: 0.8229
Epoch 9: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00009-0.81961.h5
17/23 [=====================>........] - ETA: 57s - loss: 0.7676 - categorical_accuracy: 0.8196 
Epoch 9: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00009-0.82778.h5
18/23 [======================>.......] - ETA: 47s - loss: 0.7672 - categorical_accuracy: 0.8278
Epoch 9: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00009-0.83333.h5
19/23 [=======================>......] - ETA: 38s - loss: 0.7608 - categorical_accuracy: 0.8333
Epoch 9: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00009-0.83167.

 9/23 [==========>...................] - ETA: 2:08 - loss: 0.7104 - categorical_accuracy: 0.8444
Epoch 11: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00011-0.84000.h5
10/23 [============>.................] - ETA: 1:59 - loss: 0.7150 - categorical_accuracy: 0.8400
Epoch 11: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00011-0.83636.h5
11/23 [=============>................] - ETA: 1:49 - loss: 0.7106 - categorical_accuracy: 0.8364
Epoch 11: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00011-0.84167.h5
12/23 [==============>...............] - ETA: 1:40 - loss: 0.7074 - categorical_accuracy: 0.8417
Epoch 11: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00011-0.84359.h5
13/23 [===============>..............] - ETA: 1:31 - loss: 0.7075 - categorical_accuracy: 0.8436
Epoch 11: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00011-0

...

 1/23 [>.............................] - ETA: 3:21 - loss: 0.4942 - categorical_accuracy: 0.9667
Epoch 20: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00020-0.95000.h5
 2/23 [=>............................] - ETA: 3:18 - loss: 0.4849 - categorical_accuracy: 0.9500
Epoch 20: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00020-0.93333.h5
 3/23 [==>...........................] - ETA: 3:09 - loss: 0.5041 - categorical_accuracy: 0.9333
Epoch 20: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00020-0.92500.h5
 4/23 [====>.........................] - ETA: 2:57 - loss: 0.5185 - categorical_accuracy: 0.9250
Epoch 20: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00020-0.92667.h5
 5/23 [=====>........................] - ETA: 2:47 - loss: 0.5121 - categorical_accuracy: 0.9267
Epoch 20: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00020-0

11/23 [=============>................] - ETA: 1:52 - loss: 0.4476 - categorical_accuracy: 0.9485
Epoch 23: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00023-0.94167.h5
12/23 [==============>...............] - ETA: 1:43 - loss: 0.4496 - categorical_accuracy: 0.9417
Epoch 23: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00023-0.94615.h5
13/23 [===============>..............] - ETA: 1:33 - loss: 0.4468 - categorical_accuracy: 0.9462
Epoch 23: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00023-0.94762.h5
14/23 [=================>............] - ETA: 1:24 - loss: 0.4431 - categorical_accuracy: 0.9476
Epoch 23: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00023-0.94667.h5
15/23 [==================>...........] - ETA: 1:14 - loss: 0.4437 - categorical_accuracy: 0.9467
Epoch 23: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00023-0

 4/23 [====>.........................] - ETA: 2:54 - loss: 0.4243 - categorical_accuracy: 0.9417
Epoch 25: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00025-0.94667.h5
 5/23 [=====>........................] - ETA: 2:46 - loss: 0.3953 - categorical_accuracy: 0.9467
Epoch 25: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00025-0.94444.h5
 6/23 [======>.......................] - ETA: 2:36 - loss: 0.4108 - categorical_accuracy: 0.9444
Epoch 25: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00025-0.94286.h5
 7/23 [========>.....................] - ETA: 2:27 - loss: 0.4242 - categorical_accuracy: 0.9429
Epoch 25: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00025-0.94167.h5
 8/23 [=========>....................] - ETA: 2:17 - loss: 0.4297 - categorical_accuracy: 0.9417
Epoch 25: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00025-0

15/23 [==================>...........] - ETA: 1:14 - loss: 0.4544 - categorical_accuracy: 0.9444
Epoch 28: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00028-0.94375.h5
16/23 [===================>..........] - ETA: 1:05 - loss: 0.4530 - categorical_accuracy: 0.9438
Epoch 28: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00028-0.94314.h5
17/23 [=====================>........] - ETA: 56s - loss: 0.4574 - categorical_accuracy: 0.9431 
Epoch 28: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00028-0.93889.h5
18/23 [======================>.......] - ETA: 46s - loss: 0.4601 - categorical_accuracy: 0.9389
Epoch 28: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00028-0.94035.h5
19/23 [=======================>......] - ETA: 37s - loss: 0.4625 - categorical_accuracy: 0.9404
Epoch 28: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00028-0.9

 9/23 [==========>...................] - ETA: 2:09 - loss: 0.4661 - categorical_accuracy: 0.9593
Epoch 30: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00030-0.95667.h5
10/23 [============>.................] - ETA: 2:00 - loss: 0.4667 - categorical_accuracy: 0.9567
Epoch 30: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00030-0.95455.h5
11/23 [=============>................] - ETA: 1:51 - loss: 0.4638 - categorical_accuracy: 0.9545
Epoch 30: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00030-0.95556.h5
12/23 [==============>...............] - ETA: 1:42 - loss: 0.4617 - categorical_accuracy: 0.9556
Epoch 30: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00030-0.95641.h5
13/23 [===============>..............] - ETA: 1:33 - loss: 0.4615 - categorical_accuracy: 0.9564
Epoch 30: saving model to model_init_conv_lstm_resnet50_2023-11-0214_08_06.012505\model-00030-0

###### On 23rd epoch we have got best metric score as categorical accuracy = 94.72% and val_categorical_accuracy = 92%

# Lets build Conv3D model

In [43]:
batch_size = 30
print ('# batch size =', batch_size)
num_epochs = 15
print ('# num epochs =', num_epochs)
no_of_frames = 20
print ('# num of frames =', no_of_frames)
height = 120
width = 120
print ('# hight =', height, '; width =', width)
set_steps_epoch_validation(batch_size)
train_generator = generator(train_path, train_doc, batch_size, num_epochs, no_of_frames)
val_generator = generator(val_path, val_doc, batch_size, num_epochs, no_of_frames)

# batch size = 30
# num epochs = 15
# num of frames = 20
# hight = 120 ; width = 120
# steps per epoch = 23
# validation steps = 4


In [44]:
model_3D = Sequential()
model_3D.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu', input_shape=(no_of_frames,height,width,3)))
model_3D.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))) #, strides=(2,2,2)
model_3D.add(BatchNormalization())
#model_3D.add(Conv3D(64, (3,3,3), strides=(1,1,1), activation = 'relu'))
#model_3D.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model_3D.add(BatchNormalization())
model_3D.add(Dropout(0.25))

model_3D.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D.add(BatchNormalization())
#model_3D.add(Conv3D(128, (3,3,3), strides=(1,1,1), activation = 'relu'))
#model_3D.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model_3D.add(BatchNormalization())
model_3D.add(Dropout(0.25))

model_3D.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D.add(BatchNormalization())
model_3D.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D.add(BatchNormalization())
model_3D.add(Dropout(0.25))

model_3D.add(Flatten())
model_3D.add(Dense(512, kernel_regularizer=l2(0.01), activation='relu'))
model_3D.add(Dropout(0.5))
model_3D.add(Dense(5, activation='softmax'))

In [45]:
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model_3D.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_3D.summary())

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_53 (Conv3D)          (None, 20, 120, 120, 64   5248      
                             )                                   
                                                                 
 max_pooling3d_47 (MaxPooli  (None, 10, 60, 60, 64)    0         
 ng3D)                                                           
                                                                 
 batch_normalization_44 (Ba  (None, 10, 60, 60, 64)    256       
 tchNormalization)                                               
                                                                 
 dropout_28 (Dropout)        (None, 10, 60, 60, 64)    0         
                                                                 
 conv3d_54 (Conv3D)          (None, 10, 60, 60, 128)   221312    
                                                     

In [46]:
model_name = 'model_init_3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_3D-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

In [47]:
model_3D.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
    callbacks=callbacks_list, validation_data=val_generator, 
    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train ; batch size = 30 ; epochs = 15 ; no of frames = 20 ; height = 120 ; width = 120
Epoch 1/15

Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.94231-0.13333.h5
 1/23 [>.............................] - ETA: 35:42 - loss: 12.9423 - categorical_accuracy: 0.1333

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.54670-0.18333.h5
 2/23 [=>............................] - ETA: 33:42 - loss: 12.5467 - categorical_accuracy: 0.1833
Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.19357-0.26667.h5
 3/23 [==>...........................] - ETA: 32:05 - loss: 12.1936 - categorical_accuracy: 0.2667
Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.11518-0.26667.h5
 4/23 [====>.........................] - ETA: 30:29 - loss: 12.1152 - categorical_accuracy: 0.2667
Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.32907-0.25333.h5
 5/23 [=====>........................] - ETA: 28:54 - loss: 12.3291 - categorical_accuracy: 0.2533
Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.27956-0.26667.h5
 6/23 [======>.......................] - ETA: 27:19 - loss: 12.2796 - categorical_accuracy: 0.2667
Epoc

18/23 [======================>.......] - ETA: 8:08 - loss: 11.1503 - categorical_accuracy: 0.5537
Epoch 2: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00002-11.14969-0.55263.h5
19/23 [=======================>......] - ETA: 6:30 - loss: 11.1497 - categorical_accuracy: 0.5526
Epoch 2: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00002-11.15314-0.55167.h5
20/23 [=========================>....] - ETA: 4:52 - loss: 11.1531 - categorical_accuracy: 0.5517
Epoch 2: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00002-11.13673-0.55714.h5
21/23 [==========================>...] - ETA: 3:15 - loss: 11.1367 - categorical_accuracy: 0.5571
Epoch 2: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00002-11.11537-0.56364.h5
22/23 [===========================>..] - ETA: 1:37 - loss: 11.1154 - categorical_accuracy: 0.5636
Epoch 2: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00002-11.11154-0.56410.h5
23/23 [===

 6/23 [======>.......................] - ETA: 27:42 - loss: 10.6934 - categorical_accuracy: 0.6444
Epoch 6: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00006-10.66670-0.65238.h5
 7/23 [========>.....................] - ETA: 26:03 - loss: 10.6667 - categorical_accuracy: 0.6524
Epoch 6: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00006-10.63774-0.66250.h5
 8/23 [=========>....................] - ETA: 24:25 - loss: 10.6377 - categorical_accuracy: 0.6625
Epoch 6: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00006-10.57252-0.69259.h5
 9/23 [==========>...................] - ETA: 22:47 - loss: 10.5725 - categorical_accuracy: 0.6926
Epoch 6: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00006-10.53585-0.70667.h5
10/23 [============>.................] - ETA: 21:10 - loss: 10.5359 - categorical_accuracy: 0.7067
Epoch 6: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00006-10.51347-0.71818.h5
11/23

Epoch 8/15

Epoch 8: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00008-9.90387-0.90000.h5
 1/23 [>.............................] - ETA: 36:30 - loss: 9.9039 - categorical_accuracy: 0.9000
Epoch 8: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00008-10.00302-0.83333.h5
 2/23 [=>............................] - ETA: 34:19 - loss: 10.0030 - categorical_accuracy: 0.8333
Epoch 8: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00008-10.06693-0.82222.h5
 3/23 [==>...........................] - ETA: 32:40 - loss: 10.0669 - categorical_accuracy: 0.8222
Epoch 8: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00008-10.08763-0.83333.h5
 4/23 [====>.........................] - ETA: 31:04 - loss: 10.0876 - categorical_accuracy: 0.8333
Epoch 8: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00008-10.06951-0.84000.h5
 5/23 [=====>........................] - ETA: 29:27 - loss: 10.0695 - categorical_accuracy: 0.

18/23 [======================>.......] - ETA: 8:10 - loss: 9.8782 - categorical_accuracy: 0.8981
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.87167-0.90175.h5
19/23 [=======================>......] - ETA: 6:32 - loss: 9.8717 - categorical_accuracy: 0.9018
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.87215-0.90167.h5
20/23 [=========================>....] - ETA: 4:54 - loss: 9.8722 - categorical_accuracy: 0.9017
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.87953-0.89683.h5
21/23 [==========================>...] - ETA: 3:16 - loss: 9.8795 - categorical_accuracy: 0.8968
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.88496-0.89091.h5
22/23 [===========================>..] - ETA: 1:38 - loss: 9.8850 - categorical_accuracy: 0.8909
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.89062-0.88839.h5
23/23 [=============

12/23 [==============>...............] - ETA: 18:03 - loss: 9.8034 - categorical_accuracy: 0.9278
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.79939-0.92821.h5
13/23 [===============>..............] - ETA: 16:23 - loss: 9.7994 - categorical_accuracy: 0.9282
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.82142-0.92143.h5
14/23 [=================>............] - ETA: 14:45 - loss: 9.8214 - categorical_accuracy: 0.9214
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.82345-0.92000.h5
15/23 [==================>...........] - ETA: 13:07 - loss: 9.8234 - categorical_accuracy: 0.9200
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.82716-0.91875.h5
16/23 [===================>..........] - ETA: 11:28 - loss: 9.8272 - categorical_accuracy: 0.9187
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.82482-0.91765.h5
17/23 [===

 6/23 [======>.......................] - ETA: 27:55 - loss: 9.8001 - categorical_accuracy: 0.9167
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.80502-0.90476.h5
 7/23 [========>.....................] - ETA: 26:15 - loss: 9.8050 - categorical_accuracy: 0.9048
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.78769-0.91250.h5
 8/23 [=========>....................] - ETA: 24:36 - loss: 9.7877 - categorical_accuracy: 0.9125
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.78909-0.91111.h5
 9/23 [==========>...................] - ETA: 22:58 - loss: 9.7891 - categorical_accuracy: 0.9111
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.80427-0.90333.h5
10/23 [============>.................] - ETA: 21:19 - loss: 9.8043 - categorical_accuracy: 0.9033
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.80675-0.90303.h5
11/23 [===

Epoch 15/15

Epoch 15: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00015-9.72729-0.93333.h5
 1/23 [>.............................] - ETA: 37:19 - loss: 9.7273 - categorical_accuracy: 0.9333
Epoch 15: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00015-9.72624-0.91667.h5
 2/23 [=>............................] - ETA: 35:11 - loss: 9.7262 - categorical_accuracy: 0.9167
Epoch 15: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00015-9.74565-0.92222.h5
 3/23 [==>...........................] - ETA: 33:31 - loss: 9.7457 - categorical_accuracy: 0.9222
Epoch 15: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00015-9.74361-0.93333.h5
 4/23 [====>.........................] - ETA: 32:23 - loss: 9.7436 - categorical_accuracy: 0.9333
Epoch 15: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00015-9.74232-0.94000.h5
 5/23 [=====>........................] - ETA: 30:35 - loss: 9.7423 - categorical_accuracy: 0.94

#### Change in no of frames and height and width

In [135]:
batch_size = 30
print ('# batch size =', batch_size)
num_epochs = 15
print ('# num epochs =', num_epochs)
no_of_frames = 25
print ('# num of frames =', no_of_frames)
height = 84
width = 84
print ('# hight =', height, '; width =', width)
set_steps_epoch_validation(batch_size)
train_generator = generator(train_path, train_doc, batch_size, num_epochs, no_of_frames, height, width)
val_generator = generator(val_path, val_doc, batch_size, num_epochs, no_of_frames, height, width)

# batch size = 30
# num epochs = 15
# num of frames = 25
# hight = 84 ; width = 84
# steps per epoch = 23
# validation steps = 4


In [124]:
model_3D_2 = Sequential()
model_3D_2.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu', input_shape=(no_of_frames,height,width,3)))
model_3D_2.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))) #, strides=(2,2,2)
model_3D_2.add(BatchNormalization())
#model_3D_2.add(Conv3D(64, (3,3,3), strides=(1,1,1), activation = 'relu'))
#model_3D_2.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model_3D_2.add(BatchNormalization())
model_3D_2.add(Dropout(0.25))

model_3D_2.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_2.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_2.add(BatchNormalization())
#model_3D_2.add(Conv3D(128, (3,3,3), strides=(1,1,1), activation = 'relu'))
#model_3D_2.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model_3D_2.add(BatchNormalization())
model_3D_2.add(Dropout(0.25))

model_3D_2.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_2.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_2.add(BatchNormalization())
model_3D_2.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_2.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_2.add(BatchNormalization())
model_3D_2.add(Dropout(0.25))

model_3D_2.add(Flatten())
model_3D_2.add(Dense(512, kernel_regularizer=l2(0.01), activation='relu'))
model_3D_2.add(Dropout(0.5))
model_3D_2.add(Dense(5, activation='softmax'))

In [125]:
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-5, momentum=0.7, nesterov=True)
model_3D_2.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_3D_2.summary())

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_103 (Conv3D)         (None, 25, 84, 84, 64)    5248      
                                                                 
 max_pooling3d_97 (MaxPooli  (None, 12, 42, 42, 64)    0         
 ng3D)                                                           
                                                                 
 batch_normalization_94 (Ba  (None, 12, 42, 42, 64)    256       
 tchNormalization)                                               
                                                                 
 dropout_84 (Dropout)        (None, 12, 42, 42, 64)    0         
                                                                 
 conv3d_104 (Conv3D)         (None, 12, 42, 42, 128)   221312    
                                                                 
 max_pooling3d_98 (MaxPooli  (None, 6, 21, 21, 128)  

In [126]:
model_3D_2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
    callbacks=callbacks_list, validation_data=val_generator, 
    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train ; batch size = 30 ; epochs = 15 ; no of frames = 25 ; height = 84 ; width = 84
Epoch 1/15

Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.87326-0.23333.h5
 1/23 [>.............................] - ETA: 21:13 - loss: 12.8733 - categorical_accuracy: 0.2333
Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.26702-0.25000.h5
 2/23 [=>............................] - ETA: 19:58 - loss: 12.2670 - categorical_accuracy: 0.2500
Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-11.98332-0.26667.h5
 3/23 [==>...........................] - ETA: 19:02 - loss: 11.9833 - categorical_accuracy: 0.2667
Epoch 1: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00001-12.15611-0.26667.h5
 4/23 [====>.........................] - ETA: 18:06 - loss: 12.1561 - categorical_accuracy: 0.2667
Epoch 1: saving model to model_init_3

10/23 [============>.................] - ETA: 12:40 - loss: 10.4578 - categorical_accuracy: 0.6133
Epoch 4: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00004-10.44010-0.62121.h5
11/23 [=============>................] - ETA: 11:41 - loss: 10.4401 - categorical_accuracy: 0.6212
Epoch 4: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00004-10.44202-0.61667.h5
12/23 [==============>...............] - ETA: 10:43 - loss: 10.4420 - categorical_accuracy: 0.6167
Epoch 4: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00004-10.43517-0.62308.h5
13/23 [===============>..............] - ETA: 9:44 - loss: 10.4352 - categorical_accuracy: 0.6231 
Epoch 4: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00004-10.45806-0.61429.h5
14/23 [=================>............] - ETA: 8:46 - loss: 10.4581 - categorical_accuracy: 0.6143
Epoch 4: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00004-10.42031-0.62889.h5
15/23 

22/23 [===========================>..] - ETA: 58s - loss: 9.8502 - categorical_accuracy: 0.8045 
Epoch 7: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00007-9.84896-0.80543.h5
23/23 [==============================] - 1319s 57s/step - loss: 9.8490 - categorical_accuracy: 0.8054 - val_loss: 9.9892 - val_categorical_accuracy: 0.8100 - lr: 0.0010
Epoch 8/15

Epoch 8: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00008-9.75618-0.80000.h5
 1/23 [>.............................] - ETA: 21:52 - loss: 9.7562 - categorical_accuracy: 0.8000
Epoch 8: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00008-9.72223-0.78333.h5
 2/23 [=>............................] - ETA: 20:32 - loss: 9.7222 - categorical_accuracy: 0.7833
Epoch 8: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00008-9.68439-0.82222.h5
 3/23 [==>...........................] - ETA: 19:31 - loss: 9.6844 - categorical_accuracy: 0.8222
Epoch 8: saving model to model_i

17/23 [=====================>........] - ETA: 5:50 - loss: 9.6060 - categorical_accuracy: 0.8490
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.59540-0.85556.h5
18/23 [======================>.......] - ETA: 4:51 - loss: 9.5954 - categorical_accuracy: 0.8556
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.59164-0.85614.h5
19/23 [=======================>......] - ETA: 3:53 - loss: 9.5916 - categorical_accuracy: 0.8561
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.59368-0.85667.h5
20/23 [=========================>....] - ETA: 2:55 - loss: 9.5937 - categorical_accuracy: 0.8567
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.60306-0.85556.h5
21/23 [==========================>...] - ETA: 1:56 - loss: 9.6031 - categorical_accuracy: 0.8556
Epoch 9: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00009-9.59406-0.85909.h5
22/23 [=============

11/23 [=============>................] - ETA: 11:40 - loss: 9.4900 - categorical_accuracy: 0.8909
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.48585-0.89444.h5
12/23 [==============>...............] - ETA: 10:41 - loss: 9.4858 - categorical_accuracy: 0.8944
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.47948-0.90000.h5
13/23 [===============>..............] - ETA: 9:43 - loss: 9.4795 - categorical_accuracy: 0.9000 
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.48029-0.90000.h5
14/23 [=================>............] - ETA: 8:44 - loss: 9.4803 - categorical_accuracy: 0.9000
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.48511-0.90000.h5
15/23 [==================>...........] - ETA: 7:46 - loss: 9.4851 - categorical_accuracy: 0.9000
Epoch 11: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00011-9.47617-0.90208.h5
16/23 [=====

 5/23 [=====>........................] - ETA: 17:30 - loss: 9.4341 - categorical_accuracy: 0.9133
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.42838-0.90556.h5
 6/23 [======>.......................] - ETA: 16:30 - loss: 9.4284 - categorical_accuracy: 0.9056
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.42274-0.91429.h5
 7/23 [========>.....................] - ETA: 15:32 - loss: 9.4227 - categorical_accuracy: 0.9143
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.44076-0.91250.h5
 8/23 [=========>....................] - ETA: 14:35 - loss: 9.4408 - categorical_accuracy: 0.9125
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.45443-0.90000.h5
 9/23 [==========>...................] - ETA: 13:34 - loss: 9.4544 - categorical_accuracy: 0.9000
Epoch 13: saving model to model_init_3D_2023-11-0221_05_51.849318\model_3D-00013-9.48059-0.88667.h5
10/23 [===

#### Augmentation

In [57]:
import os
import cv2
import numpy as np
import vidaug.augmentors as va
def vid_augmentation(train_path, augtype):
    # Define the paths to the input and output video directories
    input_dir = train_path
    output_dir_invertcolor = 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor'
    output_dir_salt = 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_salt'
    output_dir_pepper = 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_pepper'
    output_dir_horizontalflip = 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_horizontalflip'
    k = 0
    for folderName in df.FolderName:
        k = k + 1
        #print(k,'=>',folderName)
        imgs = os.listdir(input_dir + '/' + folderName)
        #print((imgs[0]))

        frames = []
        for sample in imgs:
            imgsample = cv2.imread(input_dir + '/' + folderName + '/' + sample)
            frames.append(imgsample)

        #print(len(frames))

        # Apply the video augmentation pipeline to each frame of the video
        sometimes = lambda aug: va.Sometimes(1, aug) # Used to apply augmentor with 50% probability
        if augtype == 'invertcolor':
            seq = va.Sequential([
                                sometimes(va.InvertColor())
            ])
            output_dir = output_dir_invertcolor 
        elif augtype == 'salt':
            seq = va.Sequential([
                                sometimes(va.Salt())
            ])
            output_dir = output_dir_salt 
        elif augtype == 'pepper':
            seq = va.Sequential([
                                sometimes(va.Pepper())
            ])
            output_dir = output_dir_pepper 
        elif augtype == 'horizontalflip':
            seq = va.Sequential([
                                sometimes(va.HorizontalFlip())
            ])
            output_dir = output_dir_horizontalflip 
        else:
            print('Invalid Augmentation')
            break

        #augment the frames
        video_aug = seq(frames)

        #break
        # save the video frames
        os.umask(0)
        for i in range(len(video_aug)):
            if not os.path.exists(output_dir + '/' + folderName):
                #print(k,'=>',output_dir + '/' + folderName)
                os.makedirs(output_dir + '/' + folderName, mode=0o777, exist_ok=True)
            cv2.imwrite(output_dir + '/'  + folderName + '/' + imgs[i], video_aug[i])

        del video_aug #, video_aug2, video_aug3, video_aug4 
        del seq #, seq2, seq3, seq4
        del sometimes 

In [52]:
vid_augmentation(train_path, 'invertcolor')

1 => WIN_20180926_16_54_08_Pro_Right_Swipe_new
30
1 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_16_54_08_Pro_Right_Swipe_new
2 => WIN_20180925_18_02_58_Pro_Thumbs_Down_new
30
2 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_18_02_58_Pro_Thumbs_Down_new
3 => WIN_20180925_17_33_08_Pro_Left_Swipe_new
30
3 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_33_08_Pro_Left_Swipe_new
4 => WIN_20180925_17_51_17_Pro_Thumbs_Up_new
30
4 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_51_17_Pro_Thumbs_Up_new
5 => WIN_20180926_17_17_35_Pro_Left_Swipe_new
30
5 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_17_35_Pro_Left_Swipe_new
6 => WIN_20180926_17_30_47_Pro_Thumbs_Up_new
30
6 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_30_47_Pro_Thumbs

30
50 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_30_24_Pro_Right_Swipe_new
51 => WIN_20180907_16_08_56_Pro_Thumbs Up_new
30
51 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_16_08_56_Pro_Thumbs Up_new
52 => WIN_20180926_16_50_21_Pro_Left_Swipe_new
30
52 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_16_50_21_Pro_Left_Swipe_new
53 => WIN_20180926_17_20_47_Pro_Thumbs_Up_new
30
53 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_20_47_Pro_Thumbs_Up_new
54 => WIN_20180907_16_17_11_Pro_Left Swipe_new_Left Swipe_new
30
54 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_16_17_11_Pro_Left Swipe_new_Left Swipe_new
55 => WIN_20180926_17_05_32_Pro_Stop_new
30
55 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_05_32_Pro_Stop_new
56 => WIN_20

100 => WIN_20180925_18_04_13_Pro_Thumbs_Down_new
30
100 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_18_04_13_Pro_Thumbs_Down_new
101 => WIN_20180925_17_46_49_Pro_Left_Swipe_new
30
101 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_46_49_Pro_Left_Swipe_new
102 => WIN_20180907_15_49_56_Pro_Right Swipe_new
30
102 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_15_49_56_Pro_Right Swipe_new
103 => WIN_20180925_17_19_19_Pro_Right_Swipe_new
30
103 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_19_19_Pro_Right_Swipe_new
104 => WIN_20180925_17_50_59_Pro_Right_Swipe_new
30
104 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_50_59_Pro_Right_Swipe_new
105 => WIN_20180907_16_00_27_Pro_Stop Gesture_new
30
105 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor

30
149 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_20_55_Pro_Right_Swipe_new
150 => WIN_20180926_16_42_06_Pro_Right_Swipe_new
30
150 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_16_42_06_Pro_Right_Swipe_new
151 => WIN_20180907_16_20_32_Pro_Thumbs Down_new
30
151 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_16_20_32_Pro_Thumbs Down_new
152 => WIN_20180926_17_32_01_Pro_Thumbs_Down_new
30
152 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_32_01_Pro_Thumbs_Down_new
153 => WIN_20180926_17_33_46_Pro_Left_Swipe_new
30
153 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_33_46_Pro_Left_Swipe_new
154 => WIN_20180926_17_11_17_Pro_Stop_new
30
154 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_11_17_Pro_Stop_new
155 => WIN_20180925_1

30
197 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_16_30_24_Pro_Stop Gesture_new
198 => WIN_20180926_17_15_10_Pro_Stop_new
30
198 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_15_10_Pro_Stop_new
199 => WIN_20180907_16_04_13_Pro_Thumbs Down_new
30
199 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_16_04_13_Pro_Thumbs Down_new
200 => WIN_20180925_17_57_22_Pro_Stop_new
30
200 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_57_22_Pro_Stop_new
201 => WIN_20180926_16_45_10_Pro_Thumbs_Up_new
30
201 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_16_45_10_Pro_Thumbs_Up_new
202 => WIN_20180925_18_01_52_Pro_Stop_new
30
202 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_18_01_52_Pro_Stop_new
203 => WIN_20180926_16_55_16_Pro_Right_Swipe_new
3

...

438 => WIN_20180926_17_42_17_Pro_Thumbs_Up_new
30
438 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_42_17_Pro_Thumbs_Up_new
439 => WIN_20180925_17_18_07_Pro_Thumbs_Up_new
30
439 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_18_07_Pro_Thumbs_Up_new
440 => WIN_20180925_17_40_22_Pro_Thumbs_Down_new
30
440 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_40_22_Pro_Thumbs_Down_new
441 => WIN_20180907_16_25_52_Pro_Thumbs Down_new
30
441 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_16_25_52_Pro_Thumbs Down_new
442 => WIN_20180926_17_33_51_Pro_Thumbs_Up_new
30
442 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_33_51_Pro_Thumbs_Up_new
443 => WIN_20180925_17_24_49_Pro_Left_Swipe_new
30
443 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_2018092

486 => WIN_20180925_17_48_24_Pro_Left_Swipe_new
30
486 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_48_24_Pro_Left_Swipe_new
487 => WIN_20180907_15_54_27_Pro_Thumbs Down_new
30
487 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_15_54_27_Pro_Thumbs Down_new
488 => WIN_20180926_16_51_55_Pro_Left_Swipe_new
30
488 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_16_51_55_Pro_Left_Swipe_new
489 => WIN_20180926_16_39_35_Pro_Stop_new
30
489 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_16_39_35_Pro_Stop_new
490 => WIN_20180926_17_37_02_Pro_Stop_new
30
490 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_37_02_Pro_Stop_new
491 => WIN_20180907_15_44_13_Pro_Stop Gesture_new
30
491 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_15_44_13_Pro_Sto

30
535 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_15_56_12_Pro_Thumbs Down_new
536 => WIN_20180926_17_21_24_Pro_Thumbs_Down_new
30
536 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_21_24_Pro_Thumbs_Down_new
537 => WIN_20180925_17_37_17_Pro_Thumbs_Up_new
30
537 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_37_17_Pro_Thumbs_Up_new
538 => WIN_20180925_17_36_58_Pro_Right_Swipe_new
30
538 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_36_58_Pro_Right_Swipe_new
539 => WIN_20180925_18_02_52_Pro_Stop_new
30
539 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_18_02_52_Pro_Stop_new
540 => WIN_20180926_16_52_38_Pro_Left_Swipe_new
30
540 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_16_52_38_Pro_Left_Swipe_new
541 => WIN_20180926_17_37

30
584 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_16_33_26_Pro_Thumbs Up_new
585 => WIN_20180907_15_57_30_Pro_Stop Gesture_new
30
585 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_15_57_30_Pro_Stop Gesture_new
586 => WIN_20180926_17_42_11_Pro_Thumbs_Down_new
30
586 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_42_11_Pro_Thumbs_Down_new
587 => WIN_20180926_17_32_19_Pro_Thumbs_Up_new
30
587 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_32_19_Pro_Thumbs_Up_new
588 => WIN_20180926_17_28_21_Pro_Right_Swipe_new
30
588 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_28_21_Pro_Right_Swipe_new
589 => WIN_20180925_17_37_14_Pro_Stop_new
30
589 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_37_14_Pro_Stop_new
590 => WIN_20180925_17_

30
632 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_21_20_Pro_Right_Swipe_new
633 => WIN_20180926_17_43_30_Pro_Left_Swipe_new
30
633 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_43_30_Pro_Left_Swipe_new
634 => WIN_20180925_17_26_34_Pro_Right_Swipe_new
30
634 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180925_17_26_34_Pro_Right_Swipe_new
635 => WIN_20180926_17_02_27_Pro_Thumbs_Up_new
30
635 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_02_27_Pro_Thumbs_Up_new
636 => WIN_20180926_17_40_52_Pro_Right_Swipe_new
30
636 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180926_17_40_52_Pro_Right_Swipe_new
637 => WIN_20180907_16_53_36_Pro_Thumbs Down_new
30
637 => C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor/WIN_20180907_16_53_36_Pro_Thumbs Down_new
638 => WIN_

In [10]:
vid_augmentation(train_path, 'salt')

In [11]:
vid_augmentation(train_path, 'pepper')

In [279]:
vid_augmentation(train_path, 'horizontalflip')

In [35]:
def generator_aug(source_path, folder_list, batch_size, num_epochs, no_of_frames=15, height = 120, width = 120, aug_list = []):
    no_of_augs = len(aug_list)
    print( 'Source path = ',source_path,'; batch size =',batch_size,'; epochs =',num_epochs,
          '; no of frames =',no_of_frames,'; height =',height,'; width =',width,'; augmentations =',aug_list)
    if no_of_frames == 25:
        img_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,16,18,20,22,23,24,25,26,27,28,29]
    elif no_of_frames == 20:
        img_idx = [0,1,2,4,6,7,8,9,10,12,14,16,18,20,22,24,26,27,28,29]
    elif no_of_frames == 18:
        img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    elif no_of_frames == 15:
        img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28]
    else:
        print('Invalid frame count: ', no_of_frames)
        return
    
    #img_idx = [0,4,6,8,10,12,14,16,18,20,22,24]
    while True:
        t = np.random.permutation(folder_list)
        #print("Permutation=>",len(t),"=>",t[0])
        num_batches = int(len(t)/batch_size)
        print("No of batches=>",num_batches)
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size,no_of_frames,height,width,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(height,width)).astype(np.float64)
                    else:
                        image = resize(image,(height,width)).astype(np.float64)
                    
                    #if you’re working on action recognition, you may want to apply temporal augmentations such as frame sampling or time warping.
                    
                    #image = apply_random_crop(image, height-20, width-20)
                    '''
                    sometimes = lambda aug: va.Sometimes(1, aug) # Used to apply augmentor with 50% probability
                    seq = va.Sequential([
                        #sometimes(va.InvertColor()),
                        sometimes(va.Salt()),
                        sometimes(va.Pepper())
                    ])
                    image = seq(image)
                    '''
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if ((len(t)%batch_size)) != 0:
            batch_data = np.zeros((len(t)%batch_size,no_of_frames,height,width,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = io.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(height,width)).astype(np.float64)
                    else:
                        image = resize(image,(height,width)).astype(np.float64)

                    #image = apply_random_crop(image, height-20, width-20)
                    '''
                    sometimes = lambda aug: va.Sometimes(1, aug) # Used to apply augmentor with 50% probability
                    seq = va.Sequential([
                        #sometimes(va.InvertColor()),
                        sometimes(va.Salt()),
                        sometimes(va.Pepper())
                    ])
                    image = seq(image)
                    ''' 
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [54]:
batch_size = 20
print ('# batch size =', batch_size)
num_epochs = 10
print ('# num epochs =', num_epochs)
no_of_frames = 25
print ('# num of frames =', no_of_frames)
height = 120
width = 120
print ('# hight =', height, '; width =', width)
set_steps_epoch_validation(batch_size)
output_dir_invertcolor = 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor'
train_generator = generator_aug(output_dir_invertcolor, train_doc, batch_size, num_epochs, no_of_frames, height, width)
val_generator = generator_aug(val_path, val_doc, batch_size, num_epochs, no_of_frames, height, width)

# batch size = 20
# num epochs = 10
# num of frames = 25
# hight = 120 ; width = 120
# steps per epoch = 34
# validation steps = 5


In [37]:
model_3D_3 = Sequential()
model_3D_3.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu', input_shape=(no_of_frames,height,width,3)))
model_3D_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))) #, strides=(2,2,2)
model_3D_3.add(BatchNormalization())
#model_3D_3.add(Conv3D(64, (3,3,3), strides=(1,1,1), activation = 'relu'))
#model_3D_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model_3D_3.add(BatchNormalization())
model_3D_3.add(Dropout(0.25))

model_3D_3.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_3.add(BatchNormalization())
#model_3D_3.add(Conv3D(128, (3,3,3), strides=(1,1,1), activation = 'relu'))
#model_3D_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model_3D_3.add(BatchNormalization())
model_3D_3.add(Dropout(0.25))

model_3D_3.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_3.add(BatchNormalization())
model_3D_3.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_3.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_3.add(BatchNormalization())
model_3D_3.add(Dropout(0.25))

model_3D_3.add(Flatten())
model_3D_3.add(Dense(512, kernel_regularizer=l2(0.01), activation='relu'))
model_3D_3.add(Dropout(0.5))
model_3D_3.add(Dense(5, activation='softmax'))

In [38]:
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-5, momentum=0.7, nesterov=True)
model_3D_3.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_3D_3.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 25, 120, 120, 64   5248      
                             )                                   
                                                                 
 max_pooling3d_8 (MaxPoolin  (None, 12, 60, 60, 64)    0         
 g3D)                                                            
                                                                 
 batch_normalization_8 (Bat  (None, 12, 60, 60, 64)    256       
 chNormalization)                                                
                                                                 
 dropout_8 (Dropout)         (None, 12, 60, 60, 64)    0         
                                                                 
 conv3d_9 (Conv3D)           (None, 12, 60, 60, 128)   221312    
                                                      

In [42]:
model_name = 'model_init_3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_3D-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

In [55]:
model_3D_3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, callbacks=callbacks_list, 
validation_data=val_generator, validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_invertcolor ; batch size = 20 ; epochs = 10 ; no of frames = 25 ; height = 120 ; width = 120 ; augmentations = []
No of batches=> 33
Epoch 1/10

Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.49516-0.45000.h5
 1/34 [..............................] - ETA: 43:47 - loss: 11.4952 - categorical_accuracy: 0.4500

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.76004-0.47500.h5
 2/34 [>.............................] - ETA: 42:31 - loss: 11.7600 - categorical_accuracy: 0.4750
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.37823-0.58333.h5
 3/34 [=>............................] - ETA: 40:58 - loss: 11.3782 - categorical_accuracy: 0.5833
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.32494-0.56250.h5
 4/34 [==>...........................] - ETA: 39:53 - loss: 11.3249 - categorical_accuracy: 0.5625
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.43729-0.53000.h5
 5/34 [===>..........................] - ETA: 38:47 - loss: 11.4373 - categorical_accuracy: 0.5300
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.65611-0.50000.h5
 6/34 [====>.........................] - ETA: 37:52 - loss: 11.6561 - categorical_accuracy: 0.5000
Epoc

 7/34 [=====>........................] - ETA: 37:50 - loss: 10.6605 - categorical_accuracy: 0.6714
Epoch 2: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00002-10.66144-0.68125.h5
 8/34 [======>.......................] - ETA: 36:16 - loss: 10.6614 - categorical_accuracy: 0.6812
Epoch 2: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00002-10.78635-0.65000.h5
 9/34 [======>.......................] - ETA: 34:54 - loss: 10.7864 - categorical_accuracy: 0.6500
Epoch 2: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00002-10.76753-0.65500.h5
10/34 [=======>......................] - ETA: 33:31 - loss: 10.7675 - categorical_accuracy: 0.6550
Epoch 2: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00002-10.79395-0.64091.h5
11/34 [========>.....................] - ETA: 32:02 - loss: 10.7940 - categorical_accuracy: 0.6409
Epoch 2: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00002-10.75057-0.65000.h5
12/34

19/34 [===============>..............] - ETA: 20:56 - loss: 10.2457 - categorical_accuracy: 0.7895
Epoch 4: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00004-10.24060-0.79250.h5
20/34 [================>.............] - ETA: 19:33 - loss: 10.2406 - categorical_accuracy: 0.7925
Epoch 4: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00004-10.24419-0.79762.h5
21/34 [=================>............] - ETA: 18:09 - loss: 10.2442 - categorical_accuracy: 0.7976
Epoch 4: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00004-10.25407-0.79773.h5
22/34 [==================>...........] - ETA: 16:45 - loss: 10.2541 - categorical_accuracy: 0.7977
Epoch 4: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00004-10.25287-0.79783.h5
23/34 [===================>..........] - ETA: 15:22 - loss: 10.2529 - categorical_accuracy: 0.7978
Epoch 4: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00004-10.24524-0.79792.h5
24/34

31/34 [==========================>...] - ETA: 4:14 - loss: 9.9086 - categorical_accuracy: 0.8855
Epoch 6: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00006-9.90659-0.88594.h5
32/34 [===========================>..] - ETA: 2:49 - loss: 9.9066 - categorical_accuracy: 0.8859No of batches=> 33

Epoch 6: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00006-9.89842-0.88939.h5
33/34 [============================>.] - ETA: 1:24 - loss: 9.8984 - categorical_accuracy: 0.8894
Epoch 6: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00006-9.89920-0.88839.h5
34/34 [==============================] - 2853s 84s/step - loss: 9.8992 - categorical_accuracy: 0.8884 - val_loss: 13.2965 - val_categorical_accuracy: 0.1600 - lr: 5.0000e-04
Epoch 7/10

Epoch 7: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00007-9.76900-1.00000.h5
 1/34 [..............................] - ETA: 49:33 - loss: 9.7690 - categorical_accuracy: 1.0000
Epoch 7: s

 2/34 [>.............................] - ETA: 43:30 - loss: 9.8180 - categorical_accuracy: 0.9000
Epoch 8: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00008-9.83140-0.90000.h5
 3/34 [=>............................] - ETA: 41:56 - loss: 9.8314 - categorical_accuracy: 0.9000
Epoch 8: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00008-9.81563-0.90000.h5
 4/34 [==>...........................] - ETA: 40:27 - loss: 9.8156 - categorical_accuracy: 0.9000
Epoch 8: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00008-9.80054-0.90000.h5
 5/34 [===>..........................] - ETA: 39:04 - loss: 9.8005 - categorical_accuracy: 0.9000
Epoch 8: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00008-9.78776-0.90833.h5
 6/34 [====>.........................] - ETA: 37:44 - loss: 9.7878 - categorical_accuracy: 0.9083
Epoch 8: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00008-9.79411-0.90714.h5
 7/34 [=====>..

14/34 [===========>..................] - ETA: 28:12 - loss: 9.7111 - categorical_accuracy: 0.9393
Epoch 10: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00010-9.70858-0.94333.h5
15/34 [============>.................] - ETA: 26:42 - loss: 9.7086 - categorical_accuracy: 0.9433
Epoch 10: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00010-9.70046-0.94687.h5
16/34 [=============>................] - ETA: 25:14 - loss: 9.7005 - categorical_accuracy: 0.9469
Epoch 10: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00010-9.69905-0.95000.h5
17/34 [==============>...............] - ETA: 23:45 - loss: 9.6991 - categorical_accuracy: 0.9500
Epoch 10: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00010-9.71739-0.93889.h5
18/34 [==============>...............] - ETA: 22:18 - loss: 9.7174 - categorical_accuracy: 0.9389
Epoch 10: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00010-9.71401-0.93947.h5
19/34 [===

#### Using multiple augmentated data

In [89]:
def generator_aug2(source_path, folder_list, batch_size, num_epochs, no_of_frames=15, height = 120, width = 120, dataset='train'):
    aug_list = ['train_invertcolor','train_horizontalflip'] # ,'train_pepper','train_horizontalflip'
    print( 'Source path = ',source_path,'; batch size =',batch_size,'; epochs =',num_epochs,
          '; no of frames =',no_of_frames,'; height =',height,'; width =',width,'; augmentations =',aug_list)
    if no_of_frames == 25:
        img_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,14,16,18,20,22,23,24,25,26,27,28,29]
    elif no_of_frames == 20:
        img_idx = [0,1,2,4,6,7,8,9,10,12,14,16,18,20,22,24,26,27,28,29]
    elif no_of_frames == 18:
        img_idx = [0,1,2,4,6,8,10,12,14,16,18,20,22,24,26,27,28,29]
    elif no_of_frames == 15:
        img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,26,28]
    else:
        print('Invalid frame count: ', no_of_frames)
        return
    
    #img_idx = [0,4,6,8,10,12,14,16,18,20,22,24]
    while True:
        temp = np.random.permutation(folder_list)
        if dataset == 'train':
            t = ['train/'+ ele for ele in temp] 
            for augtype in aug_list:
                t_aug = [(augtype+'/'+ ele) for ele in temp]
                t = t + t_aug
        else:
            t = temp
            
        print("Permutation=>",len(t),"=>",t[0])
        num_batches = int(len(t)/batch_size)
        print("No of batches=>",num_batches)
        
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size,no_of_frames,height,width,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = io.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(height,width)).astype(np.float64)
                    else:
                        image = resize(image,(height,width)).astype(np.float64)

                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if ((len(t)%batch_size)) != 0:
            batch_data = np.zeros((len(t)%batch_size,no_of_frames,height,width,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = io.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(height,width)).astype(np.float64)
                    else:
                        image = resize(image,(height,width)).astype(np.float64)

                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 104
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 117
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 123

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

In [90]:
aug_list = ['train_invertcolor','train_horizontalflip']
temp = np.random.permutation(train_doc)
t = ['train/'+ ele for ele in temp] 
for augtype in aug_list:
    t_aug = [(augtype+'/'+ ele) for ele in temp]
    t = t + t_aug
num_train_sequences = len(t)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
steps_per_epoch = 0 
validation_steps = 0

# training sequences = 1989
# validation sequences = 100


In [91]:
batch_size = 25
print ('# batch size =', batch_size)
num_epochs = 2
print ('# num epochs =', num_epochs)
no_of_frames = 25
print ('# num of frames =', no_of_frames)
height = 120
width = 120
print ('# hight =', height, '; width =', width)
set_steps_epoch_validation(batch_size)
source_path = 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data'
train_generator = generator_aug2(source_path, train_doc, batch_size, num_epochs, no_of_frames, height, width, 'train')
val_generator = generator_aug2(val_path, val_doc, batch_size, num_epochs, no_of_frames, height, width, 'val')

# batch size = 25
# num epochs = 2
# num of frames = 25
# hight = 120 ; width = 120
# steps per epoch = 80
# validation steps = 4


In [85]:
model_3D_4 = Sequential()
model_3D_4.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu', input_shape=(no_of_frames,height,width,3)))
model_3D_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2))) #, strides=(2,2,2)
model_3D_4.add(BatchNormalization())
#model_3D_4.add(Conv3D(64, (3,3,3), strides=(1,1,1), activation = 'relu'))
#model_3D_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model_3D_4.add(BatchNormalization())
model_3D_4.add(Dropout(0.25))

model_3D_4.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_4.add(BatchNormalization())
#model_3D_4.add(Conv3D(128, (3,3,3), strides=(1,1,1), activation = 'relu'))
#model_3D_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
#model_3D_4.add(BatchNormalization())
model_3D_4.add(Dropout(0.25))

model_3D_4.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_4.add(BatchNormalization())
model_3D_4.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same', activation = 'relu'))
model_3D_4.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))
model_3D_4.add(BatchNormalization())
model_3D_4.add(Dropout(0.25))

model_3D_4.add(Flatten())
model_3D_4.add(Dense(512, kernel_regularizer=l2(0.01), activation='relu'))
model_3D_4.add(Dropout(0.5))
model_3D_4.add(Dense(5, activation='softmax'))

In [86]:
sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.001, decay=1e-5, momentum=0.7, nesterov=True)
model_3D_4.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model_3D_4.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_36 (Conv3D)          (None, 25, 120, 120, 64   5248      
                             )                                   
                                                                 
 max_pooling3d_36 (MaxPooli  (None, 12, 60, 60, 64)    0         
 ng3D)                                                           
                                                                 
 batch_normalization_36 (Ba  (None, 12, 60, 60, 64)    256       
 tchNormalization)                                               
                                                                 
 dropout_36 (Dropout)        (None, 12, 60, 60, 64)    0         
                                                                 
 conv3d_37 (Conv3D)          (None, 12, 60, 60, 128)   221312    
                                                      

In [93]:
model_name = 'model_init_3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_3D-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

In [94]:
model_3D_4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, callbacks=callbacks_list, 
validation_data=val_generator, validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  C:/Per/IIITB_Data/GestureRecognition_data/Project_data ; batch size = 25 ; epochs = 2 ; no of frames = 25 ; height = 120 ; width = 120 ; augmentations = ['train_invertcolor', 'train_horizontalflip']
Permutation=> 1989 => train/WIN_20180926_17_30_50_Pro_Stop_new;Stop_new;2

No of batches=> 79
Epoch 1/2

Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-10.95080-0.64000.h5
 1/80 [..............................] - ETA: 2:13:27 - loss: 10.9508 - categorical_accuracy: 0.6400
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.10983-0.56000.h5
 2/80 [..............................] - ETA: 2:09:57 - loss: 11.1098 - categorical_accuracy: 0.5600
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-10.93811-0.58667.h5
 3/80 [>.............................] - ETA: 2:08:30 - loss: 10.9381 - categorical_accuracy: 0.5867
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-0000

40/80 [==============>...............] - ETA: 1:09:07 - loss: 11.2604 - categorical_accuracy: 0.5210
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.25809-0.52000.h5
41/80 [==============>...............] - ETA: 1:07:20 - loss: 11.2581 - categorical_accuracy: 0.5200
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.25451-0.52000.h5
42/80 [==============>...............] - ETA: 1:05:32 - loss: 11.2545 - categorical_accuracy: 0.5200
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.25473-0.51907.h5
43/80 [===============>..............] - ETA: 1:03:45 - loss: 11.2547 - categorical_accuracy: 0.5191
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.26243-0.51455.h5
44/80 [===============>..............] - ETA: 1:01:58 - loss: 11.2624 - categorical_accuracy: 0.5145
Epoch 1: saving model to model_init_3D_2023-11-0802_42_58.359998\model_3D-00001-11.24752-0.5182

UnknownError: Graph execution error:

Detected at node PyFunc defined at (most recent call last):
<stack traces unavailable>
FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Per/IIITB_Data/GestureRecognition_data/Project_data/train_horizontalflip/WIN_20180926_16_41_01_Pro_Thumbs_Up_new'
Traceback (most recent call last):

  File "C:\Users\user\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\ops\script_ops.py", line 270, in __call__
    ret = func(*args)
          ^^^^^^^^^^^

  File "C:\Users\user\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\autograph\impl\api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "C:\Users\user\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

